In [ ]:
import sys
import numpy as np

: 

### 1. 确定决策变量和约束条件
决策变量 -> 表现型 X \
约束条件 -> 解空间 

### 2. 建立优化模型

In [227]:
def f(x1, x2):
    part1 = np.power(x1, 2) - x2
    part2 = 1 - x1
    return 100 * np.power(part1, 2) + np.power(part2, 2)

### 3. 确定编码方法

In [228]:
def to_bin(value, num):
    bin_chars = ''
    temp = value
    for i in range(num):
        bin_char = bin(temp % 2)[-1]
        temp = temp // 2
        bin_chars = bin_char + bin_chars
    return bin_chars.upper()

def encode():
    y1 = np.random.randint(0, 1024)
    y2 = np.random.randint(0, 1024)
    X1 = to_bin(y1, 10)
    X2 = to_bin(y2, 10)
    X = X1 + X2
    # return X1, X2
    return X

### 4. 确定解码方法

In [229]:
# def decode(X1, X2):
def decode(X):
    X1 = X[:10]
    X2 = X[10:]
    y1 = int(X1, 2)
    y2 = int(X2, 2)
    x1 = 4.096 * y1 / 1023 - 2.048
    x2 = 4.096 * y2 / 1023 - 2.048
    return x1, x2


In [230]:
# TEST encode decode
X = encode()
print(len(X[:10]), len(X[10:]))
decode(X)

10 10


(-0.8988778103616812, 1.6756363636363636)

### 5. 确定个体评价方法 -> 计算适应度

In [231]:
# def F(X):
def F(*args):
    # x1, x2 = decode(X)
    x1 = args[0]
    x2 = args[1]
    return f(x1, x2)

# F(0.2, 0.5)

### 6. 设计遗传算子

选择运算 -> 比例选择算子\
交叉运算 -> 单点交叉算子\
变异运算 -> 基本位变异算子\

### 7. 确定遗传算法的运行参数

In [232]:
# 群体大小 
M = 80
# 中止迭代
Maxstep = 200
# 交叉概率
Pc = 0.6
# 变异概率
Pm = 0.001

### 8. 遗传算法主体部分

In [233]:
def select_operator(fitness):
    """
    1. 计算出群体中所有个体的适应度的总和
    2. 计算每个个体的相对适应度的大小, 即每个个体被遗传到下一代的概率, 每个概率值组成一个区域, 全部区域的概率值之和为1
    3. 最后再产生一个0-1随机数, 依据该随机数出现在哪个区域确定各个个体被选中的次数
    """
    total_fit = np.sum(fitness[:,1], axis=0) 
    sele_prob = np.expand_dims(fitness[:,1]/total_fit, axis=1)
    cumu_prob = np.cumsum(sele_prob, axis=0)
    selected_indexcs = []
    for i in range(fitness.shape[0]):
        tmp = np.random.random()
        select_index = np.where(tmp < cumu_prob)[0][0]
        selected_indexcs.append(select_index)
    return selected_indexcs

# select_operator(np.array([[0, 24324], [1,34534], [2,45756], [3,54564]]))

In [234]:
def crossover_operation(selected_group, gene_length=10):
    """
    1. 先对选择出来的群体进行随机配对
    2. 随机设置交叉点位置
    3. 相互交换配对的染色体之间的部分基因
    """
    crossover_group = []
    pairs_num = selected_group.shape[0]//2
    for i in range(pairs_num):
        Pairofgenes = selected_group[np.random.choice(selected_group.shape[0], 2, replace=False), :]
        crossover_position = np.random.randint(0, gene_length)
        gene_1 = Pairofgenes[0][1][:crossover_position+1] + Pairofgenes[1][1][crossover_position+1:]
        gene_2 = Pairofgenes[1][1][:crossover_position+1] + Pairofgenes[0][1][crossover_position+1:]
        crossover_group.append(gene_1)
        crossover_group.append(gene_2)
    return crossover_group

crossover_operation(np.array([[0, "0000000000"], [1,"1111111111"], [2,"1010101010"], [3,"0101010101"]], dtype=object))

['1010111111', '1111101010', '0100101010', '1011010101']

In [235]:
def mutate_operation(crossover_operation, gene_length=10):
    """ 
    1. 确定各个个体的基因变异位置
    2. 依据某一概率将变异点的原有基因值取反
    """
    mutate_group = []
    for gene in crossover_operation:
        mutate_position = np.random.randint(0, gene_length)
        if np.random.random() < Pm:
            mutated_gene = gene[:mutate_position] + str(int(not(int(gene[mutate_position])))) + gene[mutate_position+1:]
            mutate_group.append(mutated_gene)
        else:
            mutate_group.append(gene)
    return mutate_group

# mutate_operation(["0000000000"])



In [237]:
def main_genetic():
    # 初始化初代群体的基因
    group0 = np.asarray([[index, encode()] for index in range(M)], dtype=object)
    group  = group0
    # 设置迭代步数
    step = 0

    # 开始迭代
    while (step <= Maxstep):
        # 解码
        group_decode = np.asarray([[index, decode(X)] for index, X in group], dtype=object)
        
        # 计算, 评估适应度
        fitness = np.asarray([[index, F(*xi)] for index, xi in group_decode], dtype=object)

        # 选择运算: 依据与适应度成正比的概率将适应度较高的个体遗传到下一代群体中
        selected_indexes = select_operator(fitness)
        selected_group = np.asarray([[index, group[index, 1]] for index in selected_indexes], dtype=object)
        
        # 交叉运算
        crossover_group = crossover_operation(selected_group, 10)

        # 变异运算
        mutate_group = mutate_operation(crossover_group, 10)
        print(mutate_group)
        
        # 迭代出下一代群体
        pass
        step += 1
        if step == 1:
            sys.exit(0)

main_genetic()

['11110101111101000010', '11111011001110110100', '00010011100000010001', '11101110001000111111', '00100000100110000000', '11111100111001100000', '01011001001000001010', '10100000101001100000', '11111100110110000000', '11000100001000110110', '00110011100000010001', '11100100110100111011', '11110011100000010001', '11110111011010011100', '11110011001000001010', '11011001100000010001', '00101000001101111110', '11110101100011010001', '11111110110000110111', '10011111000011010000', '11111000000000110000', '00111000001001101011', '11111100110110000000', '11111100110110000000', '11110011110100111011', '00100100100000010001', '11110100110100111011', '00100101111101000010', '11001101011000110001', '11110011000101010110', '11110010110011010000', '11111111100000010001', '11011000110110000000', '11111101001000001010', '00101101100000010001', '11110010110100100001', '11011001001000111111', '00001110001000001010', '11110011001110110100', '11111011100000010001', '00101101100000010001', '11110011100011

SystemExit: 0

#### 轮盘赌demo

In [ ]:
# 随机生成5个适应度值
X = np.random.randint(1, 10, 10)
index   = np.arange(len(X))
index_X = np.concatenate((index[:, np.newaxis], X[:, np.newaxis]), axis=1)

# 计算这五个适应度值的和
tot_X   = np.sum(X)

# 计算选择概率
sele_prob = np.expand_dims(X / tot_X, axis=1)

# 计算累计概率
cumu_prob = np.cumsum(sele_prob, axis=0)

index_X_prob = np.concatenate((index_X, sele_prob, cumu_prob), axis=1)
print(index_X_prob)

# 计算每个适应度值被选中的次数
selected_tot = []
for i in range(len(X)):
    tmp = np.random.random()
    selected_herit = np.where(tmp < index_X_prob[:, 3])[0][0]
    selected_tot.append(selected_herit)
selected_tot

[[0.         8.         0.13793103 0.13793103]
 [1.         4.         0.06896552 0.20689655]
 [2.         2.         0.03448276 0.24137931]
 [3.         9.         0.15517241 0.39655172]
 [4.         7.         0.12068966 0.51724138]
 [5.         3.         0.05172414 0.56896552]
 [6.         6.         0.10344828 0.67241379]
 [7.         7.         0.12068966 0.79310345]
 [8.         3.         0.05172414 0.84482759]
 [9.         9.         0.15517241 1.        ]]
[False False False False False False False  True  True  True]
[False False False  True  True  True  True  True  True  True]
[False False False False False False False False False  True]
[ True  True  True  True  True  True  True  True  True  True]
[False False False False False False False False False  True]
[False False False False False  True  True  True  True  True]
[False False False False False False False  True  True  True]
[False False False False False False  True  True  True  True]
[ True  True  True  True  True  T

[7, 3, 9, 0, 9, 5, 7, 6, 0, 5]

In [ ]:
import numpy as np
m = np.array([
[1, 0.1],
[2, 0.2],
[3, 0.3],
[4, 0.4],
])
np.cumsum(m,axis=0)

array([[ 1. ,  0.1],
       [ 3. ,  0.3],
       [ 6. ,  0.6],
       [10. ,  1. ]])

In [ ]:
x="0"
not(int(x))

True